# Proyecto MusicStream – Años 2010- 2020

Extracción de datos desde Spotify y Last.fm

## 1. Importaciones y configuración de librerías

In [2]:
# ===============================
# LIBRERÍAS INCORPORADAS EN PYTHON
# ===============================

import os                     
# Permite a Python comunicarse con el sistema operativo
# Lo usamos para leer variables guardadas fuera del código

import time                  
# Permite hacer pausas entre peticiones
# Control de ritmo → evitar límites de la API (rate limits)

from urllib.parse import quote   
# Convierte textos con espacios o acentos en un formato válido para URLs
# Ej: "Red Hot Chili Peppers" → "Red%20Hot%20Chili%20Peppers"

# ===============================
# LIBRERÍAS EXTERNAS (instalar con pip install)
# ===============================
from dotenv import load_dotenv 
# Carga un archivo .env donde guardamos datos sensibles
# (contraseñas, claves de API) sin escribirlos directamente en el código

import requests              
# Librería general para hacer peticiones a APIs que no tienen librería propia

import spotipy               
# Librería que nos facilita hablar con la API de Spotify
# Evita que tengamos que construir peticiones HTTP a mano

from spotipy.oauth2 import SpotifyClientCredentials  
# Se usa para identificarnos ante Spotify usando credenciales
# Es como mostrar un carnet para que Spotify sepa quién hace la petición

from spotipy.exceptions import SpotifyException       
# Nos permite detectar y manejar errores cuando Spotify no responde bien

import pandas as pd          
# Herramienta principal para trabajar con tablas de datos (DataFrames)

import numpy as np           
# Ayuda a manejar valores vacíos (NaN) y tipos compatibles con bases de datos

# ===============================
# CONEXIÓN A BASE DE DATOS
# ===============================
import mysql.connector      
# Permite conectar Python con una base de datos MySQL

from mysql.connector import Error  
# Sirve para capturar errores y evitar que el programa se rompa sin avisar

# ===============================
# CARGA DE VARIABLES DE ENTORNO
# ===============================
load_dotenv()  
# Lee el archivo .env y carga sus valores en el entorno del sistema

MYSQL_HOST = os.getenv("MYSQL_HOST")        
# Dirección del servidor donde está la base de datos

MYSQL_USER = os.getenv("MYSQL_USER")        
# Usuario con permiso para acceder a la base de datos

MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
# Contraseña de la base de datos (nunca debe escribirse directamente en el código)


## 2. Credenciales y conexión Spotify

In [30]:
# --- Paso 1: Cómo obtener Client ID y Client Secret (Spotify) y qué es OAuth ---
# Usamos OAuth (Open Authorization) para poder acceder a datos públicos de Spotify (artistas, álbumes, playlists públicas)

# --- Paso 2: Cómo obtener Client ID y Client Secret (Spotify) ---
# 1️⃣ Ve a https://developer.spotify.com/dashboard e inicia sesión
# 2️⃣ Haz clic en "Create an App"
# 3️⃣ Completa los campos:
#    - App name: ej. "Mi Analizador de Música"
#    - App description: ej. "Proyecto educativo para analizar datos de canciones"
#    - Marca las casillas según cómo usarás la API (ej. "I only use the Web API")
# 4️⃣ Haz clic en "Create"
# 5️⃣ Copia el Client ID y haz clic en "SHOW CLIENT SECRET" para ver el secreto
# 6️⃣ Guárdalos en un archivo llamado ".env" en la raíz del proyecto:
#    SPOTIPY_CLIENT_ID=tu_id_aquí
#    SPOTIPY_CLIENT_SECRET=tu_secreto_aquí
# 7️⃣ CREA UN ARCHIVO LLAMADO ".gitignore" EN LA RAÍZ DEL PROYECTO Y AÑADE:
#    .env
#    Esto evita que Git suba tus credenciales a GitHub por accidente

# --- Paso 3: Recuperar las credenciales desde el archivo .env ---
client_id = os.getenv("SPOTIFY_CLIENT_ID")          # Identificador único de tu app
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")  # Clave secreta de tu app

# --- Paso 3: Validación de credenciales ---
if not client_id or not client_secret:
    raise ValueError(
        "❌ Las credenciales de Spotify no están cargadas.\n"
        "Sigue los pasos detallados arriba para obtener Client ID y Client Secret "
        "y guardarlos en tu archivo .env."
    )

# --- Paso 4: Inicialización ---
# Objetivo:
#   - Detectar si hay algún problema con las credenciales

try:
    # usamos las credenciales que permiten a nuestra app acceder a Spotify
    mis_credenciales = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    
    # Cada consulta a la API usará automáticamente nuestras credenciales
    spotify = spotipy.Spotify(auth_manager=mis_credenciales)
    
    # Confirmación visual de que todo está listo
    print("✅ Conexión con Spotify inicializada correctamente.")
    print("🔹 Listo para buscar artistas, álbumes y playlists públicas.")

except spotipy.exceptions.SpotifyException as e:
    # Si falla la conexión o las credenciales son incorrectas, mostramos el error
    raise ConnectionError(f"❌ Error al conectar con Spotify: {e}")


✅ Conexión con Spotify inicializada correctamente.
🔹 Listo para buscar artistas, álbumes y playlists públicas.


In [31]:
# ===============================
# FLUJO PROFESIONAL DETALLADO: SPOTIFY API
# ===============================

# OBJETIVO GENERAL:
# Aprender a usar la API de Spotify como lo haría un/a analista de datos:
#
#   1️⃣ Entender cómo Spotify organiza la información
#   2️⃣ Usar SEARCH solo para descubrir IDs
#   3️⃣ Usar endpoints específicos con esos IDs
#
# Idea clave:
# 👉 SEARCH = herramienta de descubrimiento
# 👉 ENDPOINTS = extracción real de datos para análisis

# ------------------------------------------------
# 1️⃣ ENTENDER LA API DE SPOTIFY (MODELO MENTAL)
# ------------------------------------------------
# Spotify NO es un buscador libre como Google.
# Es una base de datos estructurada con relaciones claras.
#
# Los principales recursos son:
#   - Artistas
#   - Álbumes / Singles
#   - Canciones (tracks)
#   - Playlists
#
# Relación jerárquica principal:
#
#   ARTISTA
#      ↓
#   ÁLBUM
#      ↓
#   CANCIONES
#
# Cada recurso tiene un ID único que Spotify usa internamente.
# Ejemplos reales:
#
#   Artista  → Shakira  → '0TnOYISbd1XYRBk9myaseg'
#   Álbum    → El Dorado → '3jjujdWJ72nww5eGnfs2E7'
#   Canción  → Chantaje → '0k17h0D3J5VfsdmQ1iZtE9'
#
# PROBLEMA REAL:
# 👉 Normalmente NO conocemos estos IDs al empezar
#
# SOLUCIÓN:
# 👉 Usamos search() SOLO para obtener esos IDs

# ------------------------------------------------
# 2️⃣ SEARCH(): DESCUBRIR IDs Y FILTRAR RESULTADOS
# ------------------------------------------------
# search() es el punto de entrada más común cuando:
#   - Tenemos texto (nombre de artista, álbum, canción)
#   - NO tenemos un ID todavía
#
# El parámetro más importante es `q`
#
# Dentro de `q` podemos usar filtros oficiales de Spotify:
#
#   album       → nombre del álbum
#   artist      → nombre del artista
#   track       → nombre de la canción
#   year        → año o rango (ej. 2010 o 2010-2015)
#   genre       → género musical (latin, jazz, rock, etc.)
#   upc         → identificador comercial de álbum
#   isrc        → identificador internacional de canción
#   tag:hipster → contenido alternativo / menos comercial
#   tag:new     → lanzamientos recientes
#
# IMPORTANTE:
# `q` puede combinar filtros:
#   q='artist:Shakira year:2010'


# Además de `q`, search() acepta parámetros de control:
#
#   type   → qué tipo de recurso queremos:
#            'artist', 'album', 'track', 'playlist'
#
#   market → país de referencia (US, ES, MX...)
#            afecta disponibilidad y popularidad
#
#   limit  → cuántos resultados devuelve (1–50)
#            para análisis solemos usar pocos y luego paginar
#
#   offset → desde qué posición empezar
#            se usa para recorrer resultados grandes
#
#   include_external → incluir contenido externo (normalmente NO se usa)
#
# En la práctica profesional:
# 👉 SEARCH se usa para localizar el recurso correcto
# 👉 Luego se abandona y se usan endpoints directos


# ------------------------------------------------
# 3️⃣ OBTENER EL ID Y DATOS DEL ARTISTA
# ------------------------------------------------
# CASO REAL:
# "Quiero analizar a un artista, pero solo conozco su nombre"

nombre_artista = "Shakira"  # Puede ser cualquier artista

resultado = spotify.search(
    q=f'artist:{nombre_artista}',  # Filtro principal: nombre del artista
    type='artist',                 # Buscamos SOLO artistas
    market='US',                   # Mercado de referencia
    limit=1,                       # Solo el resultado más relevante
    offset=0                       # Inicio de resultados
)

# Extraemos el primer artista encontrado
# (Spotify ya ordena por relevancia)
artista = resultado['artists']['items'][0]

# ID único del artista (CLAVE para siguientes pasos)
artist_id = artista['id']

# ------------------------------------------------
# 4️⃣ INTERPRETAR LOS DATOS DEL ARTISTA
# ------------------------------------------------
# Aquí NO estamos analizando canciones todavía,
# solo contexto del artista.

print("✅ Artista encontrado:", artista['name'])
print("🆔 ID del artista:", artist_id)

# Géneros asociados por Spotify (pueden ser múltiples)
print("🎼 Géneros:", artista['genres'])

# Popularidad:
# - Escala de 0 a 100
# - Calculada internamente por Spotify
# - Basada en reproducciones recientes y tendencia
print("🔥 Popularidad:", artista['popularity'])  
# Referencia común:
#   >70  → artista muy popular
#   >85  → artista global / mainstream

# Followers:
# - Número real de usuarios que siguen al artista
# - Métrica acumulada (no temporal)
print("👥 Seguidores:", artista['followers']['total'])
# Interpretación orientativa:
#   >1.000.000    → artista popular
#   >10.000.000   → superestrella
#   >50.000.000   → icono global


✅ Artista encontrado: Shakira
🆔 ID del artista: 0EmeFodog0BfCgMzAIvKQp
🎼 Géneros: ['latin pop']
🔥 Popularidad: 88
👥 Seguidores: 40816211


# 3. Extracción de datos Spotify

In [32]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA HACER SEARCH EN SPOTIFY
# ────────────────────────────────────────────────
#
# CONTEXTO:
# - search() es el punto de entrada más común a la API de Spotify
# - Se usa principalmente para:
#     👉 descubrir IDs (artistas, álbumes, canciones)
#     👉 cuando NO conocemos aún el ID exacto
#
# PROBLEMA REAL:
# - Spotify impone límites de peticiones (rate limit)
# - Si hacemos muchas búsquedas seguidas, puede devolver error 429
#
# SOLUCIÓN:
# - Envolver search() en una función "segura"
# - Detectar el error 429
# - Esperar el tiempo indicado por Spotify
# - Reintentar automáticamente


def spotify_search_seguro(spotify, query, tipo, limite=10, reintentos=3):
    """
    Realiza una búsqueda segura en la API de Spotify usando search().

    Maneja automáticamente:
    - Límite de peticiones (error 429)
    - Errores de Spotify
    - Errores inesperados del programa

    Parámetros:
    - spotify: objeto cliente de Spotipy (ya autenticado)
    - query: texto de búsqueda (ej. 'artist:Shakira', 'genre:latin year:2018')
    - tipo: tipo de recurso a buscar ('artist', 'album', 'track', 'playlist')
    - limite: número máximo de resultados a devolver (1–50)
    - reintentos: número máximo de reintentos si Spotify devuelve error 429

    Retorna:
    - Respuesta de spotify.search() si la búsqueda es exitosa
    - None si ocurre un error no recuperable
    """

    intento = 0  # Contador de intentos realizados

    # Bucle de reintentos: se repite solo si Spotify pide esperar (429)
    while intento < reintentos:
        try:
            # Llamada principal a la API de Spotify
            # Aquí es donde realmente se consulta el endpoint /search
            return spotify.search(
                q=query,
                type=tipo,
                limit=limite
            )

        except SpotifyException as e:
            # Errores específicos devueltos por la API de Spotify
            if e.http_status == 429:
                # Error 429 = demasiadas peticiones en poco tiempo
                # Spotify indica cuánto tiempo debemos esperar
                espera = int(e.headers.get("Retry-After", 1))

                print(f"⏳ Rate limit alcanzado. Esperando {espera} segundos...")
                time.sleep(espera)   # Pausa obligatoria
                intento += 1         # Incrementamos el contador de intentos
            else:
                # Otros errores de Spotify (credenciales, endpoint incorrecto, etc.)
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Cualquier error que NO venga directamente de Spotify
            # (errores de red, errores de código, etc.)
            print("❌ Error inesperado:", e)
            return None

    # Si se supera el número máximo de reintentos permitidos
    print("⚠️ Demasiados intentos fallidos. Se omite esta búsqueda.")
    return None


In [33]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA OBTENER CANCIONES DE UN ÁLBUM
# ────────────────────────────────────────────────
#
# CONTEXTO REAL EN LA API:
# - En Spotify, las canciones NO se piden directamente por artista
# - El flujo correcto es:
#
#     ARTISTA → ÁLBUM → CANCIONES
#
# - Una vez tenemos el ID de un álbum, usamos el endpoint:
#     spotify.album_tracks()
#
# PROBLEMAS REALES QUE RESUELVE ESTA FUNCIÓN:
# 1️⃣ Spotify devuelve las canciones paginadas (máx. 50 por llamada)
# 2️⃣ Spotify puede bloquear peticiones si hacemos demasiadas (error 429)
# 3️⃣ No queremos que el programa se rompa durante un análisis largo


def spotify_album_tracks_seguro(spotify, id_album, reintentos=3):
    """
    Obtiene todas las canciones de un álbum de Spotify de forma segura.

    La función:
    - Recorre automáticamente todas las páginas del álbum (paginación)
    - Detecta el error 429 (rate limit)
    - Espera el tiempo indicado por Spotify
    - Reintenta sin romper el flujo del análisis

    Parámetros:
    - spotify: cliente autenticado de Spotipy
    - id_album: ID único del álbum (obtenido previamente con search())
    - reintentos: número máximo de reintentos si Spotify bloquea la petición

    Retorna:
    - Lista completa de tracks del álbum si todo va bien
    - None si ocurre un error no recuperable
    """

    intento = 0  # Contador de reintentos por rate limit

    # Bucle de reintentos (solo se repite si Spotify pide esperar)
    while intento < reintentos:
        try:
            # Aquí se almacenarán TODAS las canciones del álbum
            tracks = []

            # Spotify devuelve como máximo 50 canciones por petición
            offset = 0
            limit = 50

            # Bucle de paginación: recorremos todas las páginas del álbum
            while True:
                # Llamada al endpoint específico del álbum
                response = spotify.album_tracks(
                    id_album,
                    limit=limit,
                    offset=offset
                )

                # Añadimos las canciones de esta página a la lista total
                tracks.extend(response['items'])

                # Si no hay más páginas, terminamos
                if response['next'] is None:
                    break

                # Avanzamos el offset para pedir la siguiente página
                offset += len(response['items'])

            # Si llegamos aquí, el álbum se descargó completo
            return tracks

        except SpotifyException as e:
            # Errores devueltos directamente por la API de Spotify
            if e.http_status == 429:
                # Error 429 = demasiadas peticiones en poco tiempo
                espera = int(e.headers.get("Retry-After", 1))

                print(f"⏳ Rate limit en álbum. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1  # Reintentamos después de esperar
            else:
                # Otros errores de Spotify (ID inválido, permisos, etc.)
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Errores inesperados (red, código, datos corruptos)
            print("❌ Error inesperado:", e)
            return None

    # Si Spotify bloquea demasiadas veces seguidas
    print("⚠️ Demasiados intentos con este álbum. Se omite el álbum.")
    return None


In [ ]:
# ────────────────────────────────────────────────
# FUNCIÓN DE NEGOCIO: BUSQUEDA DE CANCIONES EN SPOTIFY
# ────────────────────────────────────────────────

def busqueda_spotify(generos, año, spotify):
    """
    Función que busca canciones en Spotify de forma estructurada:
    flujo lógico → ARTISTA → ÁLBUM → CANCIONES

    Lógica profesional:
    1️⃣ Buscar artistas según el género.
    2️⃣ Para cada artista, buscar sus álbumes publicados en el año indicado.
    3️⃣ Para cada álbum, obtener todas las canciones.
    4️⃣ Guardar información relevante: canción, álbum, artista, género y año.

    Parámetros:
    - generos: lista de géneros musicales (ej. ["latin", "rock"])
    - año: año de lanzamiento de los álbumes a buscar
    - spotify: objeto autenticado de la API de Spotify

    Retorna:
    - DataFrame de pandas con todas las canciones encontradas
    """

    todas_las_canciones = []      # Lista donde guardaremos todas las canciones
    albumes_ya_vistos = set()     # Evita procesar álbumes duplicados

    print(f"Buscando canciones del año {año}...\n")

    for genero in generos:
        print(f"🎯 Género: {genero}")

        # -----------------------------
        # 1️⃣ Buscar artistas por género
        # -----------------------------
        # Usamos la función segura para evitar errores de rate limit
        resultado_artistas = spotify_search_seguro(
            spotify, 
            query=f"genre:{genero}",  # Filtra artistas por género
            tipo="artist", 
            limite=50                 # Máximo 50 artistas por género
        )
        if resultado_artistas is None:
            print("⚠️ No se pudieron obtener artistas de este género. Saltando...")
            continue

        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]
            artist_id = artista["id"]

            # -----------------------------
            # 2️⃣ Buscar álbumes del artista en el año
            # -----------------------------
            # No podemos filtrar artista + año directamente, hay que buscar álbumes
            busqueda_albumes = f"artist:{nombre_artista} year:{año}"
            resultado_albumes = spotify_search_seguro(
                spotify,
                query=busqueda_albumes,
                tipo="album",
                limite=50
            )
            if resultado_albumes is None:
                continue

            albumes = resultado_albumes["albums"]["items"]

            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]

                # Evitar duplicados
                if id_album in albumes_ya_vistos:
                    continue
                albumes_ya_vistos.add(id_album)

                # -----------------------------
                # 3️⃣ Obtener canciones del álbum
                # -----------------------------
                # Cada canción está asociada al álbum y artista
                resultado_canciones = spotify_album_tracks_seguro(spotify, id_album)
                if resultado_canciones is None:
                    continue

                for cancion in resultado_canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print(f"  → Canciones de este género añadidas.\n")

    # -----------------------------
    # 4️⃣ Resumen final
    # -----------------------------
    print("📊 RESUMEN:")
    for genero in generos:
        contador = sum(1 for c in todas_las_canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")
    print("Total de canciones encontradas:", len(todas_las_canciones))

    # Devolver como DataFrame para análisis
    return pd.DataFrame(todas_las_canciones)


In [ ]:
# LLAMADA A LA FUNCION PARA EXTRAER DATOS DE SPOTIFY
genero = ["country","latin","jazz","rock"] # Lista de géneros musicales a analizar
año = 2020 # cambiamos este valor para cada año seleccionado
canciones_2020_df = busqueda_spotify(genero,año, spotify) # Ejecuta la búsqueda en Spotify y devuelve los resultados en un DataFrame

Buscando canciones del año 2020
Género: country
  → Canciones de este género añadidas

Género: latin


Your application has reached a rate/request limit. Retry will occur after: 1 s
Your application has reached a rate/request limit. Retry will occur after: 1 s


  → Canciones de este género añadidas

Género: jazz


Your application has reached a rate/request limit. Retry will occur after: 85424 s


In [ ]:
canciones_2020_df  # Muestra el contenido completo del DataFrame


,nombre,artista,album,genero,año
0,Heaven - Acoustic,Kane Brown,Heaven (Acoustic),country,2018
1,Weekend,Kane Brown,Weekend,country,2018
2,Lose It - Acoustic,Kane Brown,Weekend,country,2018
3,Title Theme,HARDY,Hidden: Series One (Original Soundtrack),country,2018
4,Innocent Children,HARDY,Hidden: Series One (Original Soundtrack),country,2018
...,...,...,...,...,...
10330,That's All Right (Live Radio KWKH 16th October...,Elvis Presley,The Studio Recordings 1954,rock,2018
10331,Blue Moon Of Kentucky (Live Radio KWKH 16th Oc...,Elvis Presley,The Studio Recordings 1954,rock,2018
10332,I'll Never Stand In Your Way (Memphis Recordin...,Elvis Presley,The Studio Recordings 1954,rock,2018
10333,(It Wouldn't Be The Same) Without You [Memphis...,Elvis Presley,The Studio Recordings 1954,rock,2018


In [ ]:
canciones_2020_df.head() # Muestra las primeras filas del DataFrame para inspeccionar la estructura

,nombre,artista,album,genero,año
0,Heaven - Acoustic,Kane Brown,Heaven (Acoustic),country,2018
1,Weekend,Kane Brown,Weekend,country,2018
2,Lose It - Acoustic,Kane Brown,Weekend,country,2018
3,Title Theme,HARDY,Hidden: Series One (Original Soundtrack),country,2018
4,Innocent Children,HARDY,Hidden: Series One (Original Soundtrack),country,2018


In [ ]:
canciones_2020_df.info() # Muestra información estructural del DataFrame (columnas, tipos y nulos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10335 entries, 0 to 10334
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   nombre   10335 non-null  object
 1   artista  10335 non-null  object
 2   album    10335 non-null  object
 3   genero   10335 non-null  object
 4   año      10335 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 403.8+ KB


In [ ]:
canciones_2020_df.isnull().sum() # Cuenta los valores nulos (NaN) por columna

nombre     0
artista    0
album      0
genero     0
año        0
dtype: int64

# 4. Credenciales y conexión LAST.FM

In [4]:
# --- COMPROBACIÓN DE LA CLAVE DE LAST.FM ---
# Recupera la API key desde el archivo .env
api_key_lastfm = os.getenv("API_KEY_LASTFM")

# Verifica que la clave exista antes de usarla
if not api_key_lastfm:
    print("⚠️ ADVERTENCIA: La clave de Last.fm no está cargada. Revisa tu archivo .env.")
else:
    # Confirmamos que la clave está disponible para futuras peticiones
    print("✅ Conexión con Last.fm inicializada.")

✅ Conexión con Last.fm inicializada.


In [5]:
# ────────────────────────────────────────────────
# PRUEBA DE CONEXIÓN BÁSICA CON LAST.FM
# ────────────────────────────────────────────────

# Verificamos que la clave API de Last.fm esté definida
if not api_key_lastfm:
    print("❌ Clave API de Last.fm no definida. Revisa tu archivo .env o configuración.")
else:
    try:
        # Realizamos una petición mínima para comprobar la conexión
        # Usamos el método 'chart.getTopArtists' para obtener los artistas más populares
        respuesta = requests.get(
            "http://ws.audioscrobbler.com/2.0/",
            params={
                'method': 'chart.gettopartists',  # Endpoint de Last.fm para top artistas
                'api_key': api_key_lastfm,         # Nuestra clave API
                'format': 'json',                  # Formato de respuesta
                'limit': 1                         # Solo necesitamos 1 artista para probar
            }
        )

        # Lanza excepción si hubo error HTTP
        respuesta.raise_for_status()

        # Confirmación visual de que la conexión funciona
        print("✅ Conexión exitosa con Last.fm")
        # Nota: aquí podríamos extraer datos básicos, como el nombre del artista
        # artista_prueba = respuesta.json()['artists']['artist'][0]['name']

    except requests.exceptions.RequestException as e:
        # Captura cualquier error de la petición HTTP (conexión, timeout, clave incorrecta, etc.)
        print(f"❌ Error al conectar con Last.fm: {e}")


✅ Conexión exitosa con Last.fm


# 5. Extracción de datos LAST.FM

In [26]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/") # URL base de la API de Last.fm para realizar consultas

In [ ]:
# =======================================================
# 🔹 CONSULTA DE INFORMACIÓN DE ARTISTAS EN LAST.FM
# =======================================================

# Función que consulta Last.fm de manera segura para un artista
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    """
    Busca información de un artista en Last.fm.

    Flujo:
    1️⃣ Codifica el nombre del artista para usarlo en una URL
    2️⃣ Prepara los parámetros de la consulta (método, clave, formato)
    3️⃣ Hace la petición HTTP a la API de Last.fm
    4️⃣ Procesa la respuesta:
        - Extrae resumen de biografía (sin enlaces HTML)
        - Extrae número de oyentes ('listeners')
    5️⃣ Maneja errores de red o datos no encontrados

    Parámetros:
    - nombre_artista: str, nombre del artista
    - api_key_lastfm: str, clave de API de Last.fm

    Retorna:
    - Diccionario con:
        - 'bio_resumen': resumen textual de la biografía
        - 'listeners': número total de oyentes
        - 'consulta_exitosa': True/False
        - 'error_lastfm': mensaje de error si falla
    """
    # ───────────────────────────────
    # 1️⃣ Codificar nombre del artista
    # Convierte espacios y caracteres especiales en formato válido para URLs
    artista_codificado = quote(nombre_artista)

    # ───────────────────────────────
    # 2️⃣ Preparar parámetros de la consulta
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"
    params_info = {
        'method': 'artist.getinfo',          # Método para obtener info de artista
        'artist': artista_codificado,        # Nombre codificado
        'api_key': api_key_lastfm,           # Clave API de Last.fm
        'format': 'json'                     # Formato de respuesta
    }

    try:
        # ───────────────────────────────
        # 3️⃣ Hacer la petición HTTP con timeout
        response = requests.get(url_last_fm, params=params_info, timeout=10)
        response.raise_for_status()          # Lanza error si HTTP status != 200
        data = response.json()               # Convierte respuesta JSON a diccionario

        # ───────────────────────────────
        # 4️⃣ Procesar la respuesta
        if "artist" in data:
            artista_info = data['artist']

            # Extrae resumen de biografía, eliminando enlaces HTML
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()

            # Extrae número de oyentes únicos
            listeners = int(artista_info.get('stats', {}).get('listeners', 0))

            # Retorna datos procesados
            return {
                'bio_resumen': bio_summary,
                'listeners': listeners,
                'consulta_exitosa': True,
                'error_lastfm': None
            }
        else:
            # Caso: artista no encontrado en Last.fm
            return {
                'consulta_exitosa': False,
                'error_lastfm': "No encontrado en Last.fm",
                'bio_resumen': None,
                'listeners': 0
            }

    except requests.exceptions.RequestException as e:
        # Errores de conexión o HTTP
        status_code = getattr(e.response, 'status_code', 'N/A')
        return {
            'consulta_exitosa': False,
            'error_lastfm': f"Error API ({status_code}): {e}",
            'bio_resumen': None,
            'listeners': 0
        }

    except Exception as e:
        # Errores de procesamiento inesperados
        return {
            'consulta_exitosa': False,
            'error_lastfm': f"Error de procesamiento: {e}",
            'bio_resumen': None,
            'listeners': 0
        }


# ───────────────────────────────
# 🔹 Uso seguro de la función para todos los artistas de un DataFrame
if not api_key_lastfm:
    print("❌ La clave de la API de Last.fm no está configurada.")
else:
    # Extrae artistas únicos del DataFrame principal
    artistas_unicos = canciones_2020_df['artista'].unique()
    print(f"🔹 Total de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")

    # DataFrame temporal solo con artistas
    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista'])
    print("🔹 Iniciando consultas a Last.fm...")

    # Aplica la función a cada artista
    resultados_lastfm_serie = artistas_df['artista'].apply(
        busqueda_info_artista,
        args=(api_key_lastfm,)
    )

    # Convierte la Serie de diccionarios a DataFrame
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)
    # Añade columna de artista al inicio
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)

    # Une la info de Last.fm con el DataFrame original de canciones
    df_final = pd.merge(
        canciones_2020_df,
        datos_lastfm_df,
        on='artista',
        how='left'  # Mantiene todas las canciones aunque falten datos
    )

    print("✅ Consultas a Last.fm finalizadas y datos unidos al DataFrame.")


Total de artistas únicos a consultar en Last.fm: 150

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [71]:
df_final # Muestra el contenido completo del DataFrame

,nombre,artista,album,genero,año,bio_resumen,listeners,consulta_exitosa,error_lastfm
0,"Twelfth night, Op. 42, No. 1",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
1,"To be sung on the water, Op. 42, No. 2",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
2,"The virgin martyrs, Op. 8, No. 1",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
3,"Let down the bars, O Death, Op. 8, No. 2",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
4,"Reincarnations, Op. 16: I. Mary Hynes",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
13806,Long Progression,Red Hot Chili Peppers,Strange Man / Long Progression,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN
13807,Magpies on Fire,Red Hot Chili Peppers,Magpies on Fire / Victorian Machinery,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN
13808,Victorian Machinery,Red Hot Chili Peppers,Magpies on Fire / Victorian Machinery,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN
13809,The Sunset Sleeps,Red Hot Chili Peppers,The Sunset Sleeps / Hometown Gypsy,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN


In [ ]:
# Guardar DataFrame en CSV
df_final.to_csv('canciones_2020_con_lastfm.csv', index=False, sep=';', encoding='utf-8-sig')
print("DataFrame guardado correctamente en 'canciones_2020_con_lastfm.csv'.")

DataFrame guardado correctamente en 'canciones_2018_con_lastfm.csv'.
